# Matrizes e vetores em Julia

## Criando e acessando

Para criar vetores para números reais, o comando genérico é

    Vector{Float64}(tamanho)
    
porém, existem outros comandos para criar vetores especiais: `ones`, `zeros` e `rand`.

Ao criar um vetor com o comando `Vector` recebemos apenas um espaço na memória, cheio de sujeira.

In [2]:
v = Vector{Float64}(5)

5-element Array{Float64,1}:
 6.93848e-310
 6.93848e-310
 6.93848e-310
 6.93848e-310
 6.93848e-310

In [3]:
pointer(v) # Esse e' o endereco do vetor

Ptr{Float64} @0x00007fb9e7be0450

In [4]:
sizeof(v) # Tamanho em bytes ocupado pelo vetor

40

O comando abaixo cria um vetor de zeros. Como estamos guardando em `v` novamente, perdemos o endereço criado acima.

In [5]:
v = zeros(5)

5-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0

In [6]:
pointer(v)

Ptr{Float64} @0x00007f90e1d1eb50

In [7]:
ones(5)

5-element Array{Float64,1}:
 1.0
 1.0
 1.0
 1.0
 1.0

In [8]:
rand(5)

5-element Array{Float64,1}:
 0.780046
 0.822236
 0.73462 
 0.454318
 0.369563

Para trabalhar com vetores (e matrizes também) utilizamos índices dentro de colchetes. A seguir criamos um vetor de zeros, colocamos o valor 1 na terceira posição e guardamos na quinta posição o resultado da soma de seus valores na primeira e terceira posições.

In [9]:
v = zeros(7)

# Armazena o valor 1 na posicao 3
v[3] = 1.0

# Armazena na posicao 5 o resultado da soma dos valores contidos nas posicoes 1 e 3
v[5] = v[1] + v[3]

println(v)

[0.0,0.0,1.0,0.0,1.0,0.0,0.0]


Para criar matrizes de números reais, o comando

    Array(Float64, m, n)
    
reserva um espaço para uma matriz com $m$ linhas e $n$ colunas e devolve o seu endereço na memória. Dentro dessa matriz não sabemos o que tem.

De forma análoga, temos os comandos `zeros`, `ones` e `rand` para matrizes, que criam e colocam valores na matriz criada.

In [19]:
M = Array(Float64, 5, 3) # Pode ter qualquer coisa nas posicoes da matriz criada dessa forma

5×3 Array{Float64,2}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [20]:
pointer(M)

Ptr{Float64} @0x00007fb9eb0ae270

In [21]:
sizeof(M)

120

In [6]:
M = zeros(5, 3)

5×3 Array{Float64,2}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [7]:
M = ones(5, 3)

5×3 Array{Float64,2}:
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0

In [8]:
M = rand(5, 3)

5×3 Array{Float64,2}:
 0.104878    0.00135856  0.182159 
 0.161347    0.194412    0.0372815
 0.00775358  0.146777    0.195983 
 0.945862    0.0570584   0.745698 
 0.62287     0.587827    0.93779  

A posição $(i, j)$ de uma matriz $M$ é acessada também utilizando colchetes, mas agora com duas dimensões, separadas por vírgula.

In [9]:
# Cria uma matriz de 1s
M = ones(2, 7)

# Coloca os valor 10 na posição (1,5), substituindo o valor existente
M[1, 5] = 10.0

# Imprime o resultado da soma dos elementos (1,5), (2,1) e (2,7)
soma = M[1, 5] + M[2, 1] + M[2, 7]

println("O valor da soma e' $(soma).")

O valor da soma e' 12.0.


Colunas e linhas de uma matriz podem ser acessadas usando `:`, que significa **todas**. Por exemplo,

    M[:, 2]
    
seleciona **todas as linhas** da coluna 2 de $M$. Por outro lado

    M[2, :]
    
seleciona **todas as colunas** da linha 2 de $M$.

Nos comandos abaixo, selecionamos $M^2$ e $M_2$.

In [16]:
M = rand(2, 4)
display(M)

println(M[:, 2])

println("Dimensões da coluna: $(size(M[:, 2]))\n")

println(M[2, :])

println("Dimensões da linha: $(size(M[2, :]))\n")

2×4 Array{Float64,2}:
 0.661572  0.162355  0.227034  0.420001
 0.816956  0.378835  0.492678  0.820599

[0.162355,0.378835]
Dimensões da coluna: (2,)

[0.816956,0.378835,0.492678,0.820599]
Dimensões da linha: (4,)



O comando `size` devolve as dimensões $m$, e $n$ da matriz. No caso acima, ambos são vetores com tamanhos 2 e 4, respectivamente.

**Cuidado!** O comando abaixo cria uma *cópia* da coluna de $M$, não é como se ele fosse o endereço dessa parte da matriz. No exemplo abaixo, uma modificação em `coluna` não causa uma modificação na matriz `M` (a recíproca também é verdadeira).

In [14]:
M = zeros(3, 3)

# Coluna e um vetor (coluna) com 3 elementos
coluna = M[:, 2]

coluna[1] = 2.0

display(coluna)

display(M)

M[3, 2] = - 1.0

display(M)

display(coluna)

3-element Array{Float64,1}:
 2.0
 0.0
 0.0

3×3 Array{Float64,2}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

3×3 Array{Float64,2}:
 0.0   0.0  0.0
 0.0   0.0  0.0
 0.0  -1.0  0.0

3-element Array{Float64,1}:
 2.0
 0.0
 0.0

## Percorrendo com laços

Geralmente usamos laços do tipo `for` para percorrer vetores e matrizes. A razão para isso é que geralmente sabemos seu tamanho e temos que percorrê-los por inteiro.

Os comandos abaixo *inicializam* um vetor $v$ de forma que $v_i = i^2$.

In [23]:
v = Vector{Float64}(10)

for i = 1:length(v)
    v[i] = i^2
end

println(v)

[1.0,4.0,9.0,16.0,25.0,36.0,49.0,64.0,81.0,100.0]


O comando `length` devolve o tamanho de um vetor.

Abaixo, somamos todas as posições do vetor $v$ criado acima: $\sum \limits_{i = 1}^n v_i$

In [36]:
soma = 0.0
for i = 1:length(v)
    soma = soma + v[i]
end

println("A soma e' $(soma).")

A soma e' 385.0.


Para matrizes, necessitamos de 2 laços: um para linhas e outro para colunas

In [38]:
M = Array(Float64, 5, 10)

m, n = size(M)

# Em Julia, e' mais inteligente fixar a coluna primeiro
for j = 1:n
    for i = 1:m
        M[i, j] = i + j
    end
end

display(M)

5×10 Array{Float64,2}:
 2.0  3.0  4.0  5.0   6.0   7.0   8.0   9.0  10.0  11.0
 3.0  4.0  5.0  6.0   7.0   8.0   9.0  10.0  11.0  12.0
 4.0  5.0  6.0  7.0   8.0   9.0  10.0  11.0  12.0  13.0
 5.0  6.0  7.0  8.0   9.0  10.0  11.0  12.0  13.0  14.0
 6.0  7.0  8.0  9.0  10.0  11.0  12.0  13.0  14.0  15.0

Abaixo, somamos todas as posições do vetor $M$ criado acima: $\sum \limits_{j = 1}^n \sum \limits_{i = 1}^m M_{ij}$

In [39]:
s = 0.0

for j = 1:n
    for i = 1:m
        s = s + M[i, j]
    end
end

println("A soma e' $(s)")

A soma e' 425.0


## Memória

### Acesso

A forma como acessamos matrizes e vetores na memória tem uma grande influência na eficiência do código computacional. Tudo é influenciado pela forma como a linguagem de programação armazena a matriz na memória. Além disso, existe uma memória adicional, chamada *cache* que armazena temporariamente pequenas quantidades do vetor.

Vamos ver o impacto de acessar elementos distantes na memoria em um vetor. Para isso criamos as funções abaixo, que realizam a soma dos elementos de um vetor `v`.

In [72]:
"""
Esta funcao realiza a soma dos elementos do vetor `v` de forma ineficiente
"""
function soma_lento(v)
    
    n = length(v)
    
    n2 = Int(round(n / 2))
    
    s = 0.0
   
    # Esta soma olha para 2 posicoes distantes na memoria
    for i = 1:n2
        
        s = s + v[i] + v[n2 + i]
        
    end
    
    return s
    
end

"""
Esta funcao realiza a soma dos elementos do vetor `v` aproveitando o conhecimento de que
posicoes proximas sao acessadas de forma mais rapida.
"""
function soma_rapido(v)
    
    n = length(v)
    
    s = 0.0
   
    # Soma dois valores proximos na memoria
    for i = 1:2:n
        
        s = s + v[i] + v[i + 1]
        
    end
    
    return s
    
end

soma_rapido

Para medir o tempo, primeiramente temos que chamar as funções uma vez, com um vetor pequeno, apenas para o Julia compilar eficientemente o código.

In [73]:
vetor = ones(10); # cria um vetor de 1's.

soma_lento(vetor);

soma_rapido(vetor);

Agora criamos um vetor aleatório com tamanho grande e medimos o tempo que cada função leva. **Atenção** ao criar um vetor muito grande, pois sua máquina pode travar. Quanto maior o vetor, maior a diferença entre os tempos.

In [74]:
# Cria um vetor aleatorio. O tamanho escolhido deve ser natural e par.
# Experimente aumentar o tamanho aos pouquinhos e ver como a diferença aumenta.
vetor = rand(10000000)

# Calcula o tempo para a soma ineficiente
println("Lento: ", @elapsed(soma_lento(vetor)))

# Calcula o tempo para a soma mais eficiente
println("Rapido: ", @elapsed(soma_rapido(vetor)))

Lento: 0.010197861
Rapido: 0.009762033


O comando `@elapsed` conta o tempo para executar o comando passado por argumento.

### Economizando memória

Você sabe a diferença entre os dois blocos de comando abaixo?

In [29]:
v1 = zeros(100)
#println(pointer(v1))
v1[10] = -10.0

v1 = zeros(100);
#println(pointer(v1))

Ptr{Float64} @0x00007fb9ea8334d0
Ptr{Float64} @0x00007fb9eafbc050


In [30]:
v2 = Vector{Float64}(100)

for i = 1:length(v2)
    v2[i] = 0.0
end
#println(pointer(v2))

v2[10] = -10.0

for i = 1:length(v2)
    v2[i] = 0.0
end
#println(pointer(v2))

Ptr{Float64} @0x00007fb9eafbc3d0
Ptr{Float64} @0x00007fb9eafbc3d0


Remova os comandos comentados e rode-os novamente. Observe que o endereço da memória no primeiro bloco é alterado, enquanto que no segundo não. Isso significa que toda vez que o comando `zeros` é chamado, ele cria um **novo** vetor, reservando um **novo** espaço na memória!

In [31]:
function zera_muita_memoria()
    
    for k = 1:1000
        
        v = zeros(100)
        
        v[1] = 10.0
    end
    
end

function zera_pouca_memoria()

    v = Vector{Float64}(100)
    
    for k = 1:1000
        
        for i = 1:length(v)
            v[i] = 0.0
        end
        
        v[1] = 10.0
    end
    
end    

zera_pouca_memoria (generic function with 1 method)

In [34]:
@time zera_muita_memoria()

  0.000098 seconds (1.00 k allocations: 875.156 KB)


In [35]:
@time zera_pouca_memoria()

  0.000052 seconds (5 allocations: 1.031 KB)


O comando `@time` devolve o <U>tempo gasto</U> e o <U>número de alocações</U> de espaço na memória realizadas pela função.

A primeira função utiliza aproximadamente **800** vezes mais memória que a segunda! Note que, ao invés de `Vector` poderíamos ter utilizado `zeros` na segunda função, contanto que estivesse *fora* do laço em $k$.

## Matrizes na memória

Agora que observamos que valores próximos de um elemento do vetor são acessados mais rapidamente do que valores mais distantes, podemos estender o raciocínio para matrizes.

Seja $M$ uma matriz $m \times n$ e considere o elemento $M_{i,j}$. Qual elemento você acha que está mais próximo de $M_{i,j}$?

  - $M_{i + 1, j}$
  - $M_{i, j + 1}$
  - $M_{i + 1, j + 1}$
  - Todos
  
A resposta é: depende da linguagem de programação.

Em Julia, matrizes são criadas na memória por colunas (*column oriented*, em inglês). Na linguagem de programação `C`, por exemplo, elas são criadas por linhas (*row oriented*). `Fortran` também é em colunas.

In [32]:
M = Array(Float64, 10, 10)

10×10 Array{Float64,2}:
 5.35725e199   6.0267e175    1.04e-152     …  2.13115e-312  0.0         
 1.05695e-144  2.09537e-110  1.13245e-258     0.0           0.0         
 4.45197e252   5.98142e-154  2.19993e-152     0.0           0.0         
 2.06917e161   3.24255e-86   8.96955e-66      0.0           0.0         
 1.50281e261   5.98142e-154  6.01347e-154     0.0           0.0         
 2.43566e-154  2.87513e161   6.01347e-154  …  0.0           0.0         
 6.83345e212   2.09574e214   2.28236e-152     0.0           0.0         
 5.8967e-109   1.04099e-152  5.84527e-101     0.0           0.0         
 5.98142e-154  3.67888e228   1.01157e-153     0.0           6.92113e-310
 1.81068e44    2.8752e161    6.01347e-154     0.0           6.92113e-310

O Código abaixo, simplesmente coloca números de 1 a 100 na matriz `M` criada acima. Como veremos adiante, a forma abaixo é a forma incorreta de acessar `M` em Julia.

In [34]:
k = 1.0

for i = 1:10
    
    for j = 1:10
        
        M[i, j] = k
        
        k = k + 1
        
    end
    
end

display(M)

10×10 Array{Float64,2}:
  1.0   2.0   3.0   4.0   5.0   6.0   7.0   8.0   9.0   10.0
 11.0  12.0  13.0  14.0  15.0  16.0  17.0  18.0  19.0   20.0
 21.0  22.0  23.0  24.0  25.0  26.0  27.0  28.0  29.0   30.0
 31.0  32.0  33.0  34.0  35.0  36.0  37.0  38.0  39.0   40.0
 41.0  42.0  43.0  44.0  45.0  46.0  47.0  48.0  49.0   50.0
 51.0  52.0  53.0  54.0  55.0  56.0  57.0  58.0  59.0   60.0
 61.0  62.0  63.0  64.0  65.0  66.0  67.0  68.0  69.0   70.0
 71.0  72.0  73.0  74.0  75.0  76.0  77.0  78.0  79.0   80.0
 81.0  82.0  83.0  84.0  85.0  86.0  87.0  88.0  89.0   90.0
 91.0  92.0  93.0  94.0  95.0  96.0  97.0  98.0  99.0  100.0

Para verificarmos inicialmente que matrizes são armazenadas por colunas, basta notarmos que Julia permite que acessemos os elementos de uma matriz como se ela fosse um vetor. O que você acha que é o elemento `M[15]`?

In [35]:
M[15]

42.0

Observe que interessante o que ocorre quando percorremos a *matriz* `M` como um *vetor* em Julia!

In [37]:
for i = 1:100
    println(M[i])
end

1.0
11.0
21.0
31.0
41.0
51.0
61.0
71.0
81.0
91.0
2.0
12.0
22.0
32.0
42.0
52.0
62.0
72.0
82.0
92.0
3.0
13.0
23.0
33.0
43.0
53.0
63.0
73.0
83.0
93.0
4.0
14.0
24.0
34.0
44.0
54.0
64.0
74.0
84.0
94.0
5.0
15.0
25.0
35.0
45.0
55.0
65.0
75.0
85.0
95.0
6.0
16.0
26.0
36.0
46.0
56.0
66.0
76.0
86.0
96.0
7.0
17.0
27.0
37.0
47.0
57.0
67.0
77.0
87.0
97.0
8.0
18.0
28.0
38.0
48.0
58.0
68.0
78.0
88.0
98.0
9.0
19.0
29.0
39.0
49.0
59.0
69.0
79.0
89.0
99.0
10.0
20.0
30.0
40.0
50.0
60.0
70.0
80.0
90.0
100.0


Podemo ver que o elemento $M_{10,1}$ está mais próximo, *na memória*, de $M_{1,2}$ que de $M_{10, 2}$!

In [68]:
function inicializa_por_linha(M)
    
    k = 0.0
    
    m, n = size(M)
    
    # Fixa a linha
    for i = 1:m
        
        # Percorre as colunas
        for j = 1:n
            
            M[i, j] = k
            
            k = k + 1
            
        end
        
    end
    
end

function inicializa_por_coluna(M)
    
    k = 0.0
    
    m, n = size(M)
    
    # Fixa uma coluna
    for j = 1:n
        
        # Percorre as linhas
        for i = 1:m
            
            M[i, j] = k
            
            k = k + 1
            
        end
        
    end
       
end

inicializa_por_coluna (generic function with 1 method)

In [69]:
M = zeros(10, 10);

@elapsed(inicializa_por_linha(M));

@elapsed(inicializa_por_coluna(M));

In [70]:
m = 1000
n = 1000

M = Array(Float64, m, n)

println("Por linha: ", @elapsed(inicializa_por_linha(M)))

println("Por coluna:", @elapsed(inicializa_por_coluna(M)))

Por linha: 0.002707039
Por coluna:0.00093053


**Observação**. Note que cada função gera uma matriz diferente (teste com $n$ e $m$ iguais a 10 para ver). Um exercício interessante é saber como modificar o código de colunas para produzir a mesma matriz da função por linhas (ou vice-versa).